In [1]:
import pandas as pd
import numpy as np

import time
from datetime import datetime

from random import randint

import re
import string

import nltk
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

from sklearn.utils import shuffle

from tqdm import tqdm

import json

import paralleldots
paralleldots.set_api_key("Vo6Oh77LXmw5HXMu691RKQIZG03nY1zB9rsVTkhEHqE") # lrl818@qq.com
print('Current key:', 'Vo6Oh77LXmw5HXMu691RKQIZG03nY1zB9rsVTkhEHqE', 1)



keys = ['OwoCuujTTYS5o3Vrx0M4A7RevaHLs7uUHlUD3Afa0XI' # laualan@connect.hku.hk
        'DwGe3alGduDGJnE1FT3O112wMDNJhZDnGysR0KFfZgY', # rlalan@outlook.com
        # 'JIkATWugt8LIP3PbQrdm6cfm1hZ4DbouVoyw8oAQqhI', # 394414515@qq.com
        'Vf9mBAl7wp0s8Fl43E9aDtVPOZhftIAv0pYJvIlYGD4', # 806124854@qq.com
        # '8gAwuDOEE92zakGfDXc6PoCtqhVop0htEnEd4IHFe0U', # yleun.lau@gmail.com
        'xTNXoCxtbg36jCgT6ArKgyOPJtnJdf5QQpEEKYoSwu8', # alanconstantinelau@gmail.com
        'INsEQXjCWuaZJQyzEbHg5Kw6C8liJiHf2DXOHqmHG70', # 122075300@qq.com
        'jpctHohVPtW9QlezbIw4hZ8Ftwh8kXD14U7JlR0WRQo', # 274038499@qq.com
        'qrJoLzvIQqU1ygqtzWDw2JC59zkBsPqSzLLIeRSxLHA', # 562040899@qq.com
        'UtEqIoCXWJN5XiyfgpXJ7gg23LgOu7jVQjl36Xg7UmQ', # wgj0905@hku.hk
        'sHyni9kN5Elhtrmb3Z10f5USPyyn9o0snPCzIqagafM' # liuhuan19951021@163.com
        ]

Current key: Vo6Oh77LXmw5HXMu691RKQIZG03nY1zB9rsVTkhEHqE 1


In [2]:
df = pd.read_csv('./reindex_df.csv')
print(df.shape)

(237671, 7)


In [10]:
df.head()

,index,song,year,artist,genre,lyrics,word_count
0,0,louder-flux-pavilion-doctor-p-remix,2012,dj-fresh,Electronic,I can't control this feeling Something's happe...,341.0
1,1,that-s-my-name,2009,akcent,Pop,In my heart I will keep you In my heart Foreve...,318.0
2,2,lemonade,2007,apologetix,Rock,"Oh, Mama, I've been cleared of my crimes and I...",250.0
3,3,return-of-the-hustle,2007,fabolous,Hip-Hop,Record mode! (Record mode!) Get your money in ...,681.0
4,4,so-sad,1974,george-harrison,Rock,Now the winter has come To eclipse out the sun...,163.0


In [21]:
emo_path = r'./lyrics_emotion.json'

In [22]:
with open(emo_path, 'r') as fn:
    emo_dict = json.load(fn)
  
lyric_emo = df['lyrics'].tolist()
index_list = df.index.tolist()

total = len(index_list)
error_count = 0
key_count = 0

# for i in range(0, len(df)):
#     pass
for k, v in emo_dict.items():
    randtime = randint(0, 4)
    if 'emotion' not in v:
        lyrics = df.loc[df.index == int(k)]['lyrics']
        index = df.loc[df.index == int(k)]['index']
#         print(k)
#         print(v)
#         break
        try:
            time.sleep(randtime)
            response = paralleldots.emotion(lyrics)
            if 'code' in response and key_count < len(keys):
                key_count += 1
                print('Current key:', keys[key_count], key_count + 1)
                paralleldots.set_api_key(keys[key_count])
                response = paralleldots.emotion(lyrics)
            elif key_count == len(keys):
                print('Stop at', str(index))
                update_error(str(index) + '\tStop.')
                break
            elif 'message' in response:
                printt('Stop at', str(index))
                update_error(str(index) + '\t' + str(response['message']))
                break
            elif 'Error' in response:
                print('Stop at', str(index))
                update_error(str(index) + '\t' + str(response))
                break
            elif 'emotion' not in response:
                print('Stop at', str(index))
                update_error(str(index) + '\t' + str(response))
                break
            finished_index = update_emo(index, response)
    #         print(finished_index, 'done: ', str(round(((index-1)/total)*100, 5)), str(datetime.now()))
        except Exception as e:
            update_error(str(index) + '\t' + str(e))
            error_count += 1
            raise e
    #         print(e)
#             pass
#     elif 

Current key: JIkATWugt8LIP3PbQrdm6cfm1hZ4DbouVoyw8oAQqhI 2
57591
{'Error': 'Monthy Limit Exceeded. Please upgrade your account from your user dashboard at https://user.apis.paralleldots.com/user_dashboard', 'code': 200, 'time': '2020-04-29 15:57:35.941484'}
57591 done
Current key: Vf9mBAl7wp0s8Fl43E9aDtVPOZhftIAv0pYJvIlYGD4 3
57815
{'Error': 'Daily Limit Exceeded. Please upgrade your account from your user dashboard at https://user.apis.paralleldots.com/user_dashboard', 'code': 403, 'time': '2020-04-29 15:57:40.039473'}
57815 done
Current key: 8gAwuDOEE92zakGfDXc6PoCtqhVop0htEnEd4IHFe0U 4
58970
{'Error': 'Monthy Limit Exceeded. Please upgrade your account from your user dashboard at https://user.apis.paralleldots.com/user_dashboard', 'code': 200, 'time': '2020-04-29 15:57:46.265942'}
58970 done


In [17]:
emo_path_out = r'./lyrics_emotion.json'
emo_path_error = r'lyrics_emotion_fixed_error'


def update_emo(index, emo):
    index = index.tolist()[0]
    emo['time'] = str(datetime.now())
    try:
        with open(emo_path_out, 'r') as fn:
            emo_dict = json.load(fn)
        print(index)
        print(emo)
        emo_dict[index] = emo
    except Exception as e:
        emo_dict = {}
        emo_dict[index] = emo
    with open(emo_path_out, 'w') as fp:
        json.dump(emo_dict, fp)
    print(index, 'done')
    return index

def update_error(index):
    with open(emo_path_error, 'a') as fn:
        fn.write(str(index) + '\n')

In [18]:
with open(emo_path_out, 'r') as fn:
    emo_fixed = json.load(fn)

In [23]:
for k, v in emo_fixed.items():
    if 'emotion' not in v:
        print(k)
        print(v)

57591
{'Error': 'Monthy Limit Exceeded. Please upgrade your account from your user dashboard at https://user.apis.paralleldots.com/user_dashboard', 'code': 200, 'time': '2020-04-29 11:20:29.621337'}
57815
{'Error': 'Monthy Limit Exceeded. Please upgrade your account from your user dashboard at https://user.apis.paralleldots.com/user_dashboard', 'code': 200, 'time': '2020-04-29 11:36:55.468899'}
58970
{'Error': 'Monthy Limit Exceeded. Please upgrade your account from your user dashboard at https://user.apis.paralleldots.com/user_dashboard', 'code': 200, 'time': '2020-04-29 15:09:00.524566'}


In [24]:
len(emo_fixed)

72852

In [41]:
emo_fixed['4997']

{'emotion': {'Fear': 0.2107523579,
  'Sad': 0.2591371423,
  'Bored': 0.0815402242,
  'Happy': 0.2017061049,
  'Excited': 0.0812641845,
  'Angry': 0.1655999862},
 'time': '2020-04-27 16:38:49.211029'}

In [18]:
len(emo_fixed)

70235

In [25]:
no_id = []

for i in range(len(emo_fixed)):
    try:
        error_i = emo_fixed[str(i)]
        # print(error_i)
    except Exception as e:
        print(e)
        no_id.append(i)

'58971'
'58972'
'58973'
'58974'
'58975'
'58976'
'58977'
'58978'
'58979'
'58980'
'58981'
'58982'
'58983'
'58984'
'58985'
'58986'
'58987'
'58988'
'58989'
'58990'
'58991'
'58992'
'58993'
'58994'
'58995'
'58996'
'58997'
'58998'
'58999'
'59000'
'59001'
'59002'
'59003'
'59004'
'59005'
'59006'
'59007'
'59008'
'59009'
'59010'
'59011'
'59012'
'59013'
'59014'
'59015'
'59016'
'59017'
'59018'
'59019'
'59020'
'59021'
'59022'
'59023'
'59024'
'59025'
'59026'
'59027'
'59028'
'59029'
'59030'
'59031'
'59032'
'59033'
'59034'
'59035'
'59036'
'59037'
'59038'
'59039'
'59040'
'59041'
'59042'
'59043'
'59044'
'59045'
'59046'
'59047'
'59048'
'59049'
'59050'
'59051'
'59052'
'59053'
'59054'
'59055'
'59056'
'59057'
'59058'
'59059'
'59060'
'59061'
'59062'
'59063'
'59064'
'59065'
'59066'
'59067'
'59068'
'59069'
'59070'
'59071'
'59072'
'59073'
'59074'
'59075'
'59076'
'59077'
'59078'
'59079'
'59080'
'59081'
'59082'
'59083'
'59084'
'59085'
'59086'
'59087'
'59088'
'59089'
'59090'
'59091'
'59092'
'59093'
'59094'
'59095'


In [26]:
len(no_id)

179

In [36]:
total = len(index_list)
error_count = 0
key_count = 0

# for i in range(0, len(df)):
#     pass
for k in no_id:
    randtime = randint(0, 4)
    # if 'emotion' not in v:
    lyrics = df.loc[df.index == int(k)]['lyrics']
    index = df.loc[df.index == int(k)]['index']
    print(k)
#         print(v)
#         break
    try:
        time.sleep(randtime)
        response = paralleldots.emotion(lyrics)
        if 'code' in response and key_count < len(keys):
            key_count += 1
            print('Current key:', keys[key_count], key_count + 1)
            paralleldots.set_api_key(keys[key_count])
            response = paralleldots.emotion(lyrics)
        elif key_count == len(keys):
            print('Stop at', str(index))
            update_error(str(index) + '\tStop.')
            break
        elif 'message' in response:
            printt('Stop at', str(index))
            update_error(str(index) + '\t' + str(response['message']))
            break
        elif 'Error' in response:
            print('Stop at', str(index))
            update_error(str(index) + '\t' + str(response))
            break
        elif 'emotion' not in response:
            print('Stop at', str(index))
            update_error(str(index) + '\t' + str(response))
            break
        finished_index = update_emo(index, response)
#         print(finished_index, 'done: ', str(round(((index-1)/total)*100, 5)), str(datetime.now()))
    except Exception as e:
        update_error(str(index) + '\t' + str(e))
        error_count += 1
        raise e
    #         print(e)
#             pass
#     elif 

31345, 'Bored': 0.0222318043, 'Happy': 0.6698081477, 'Excited': 0.098466264, 'Angry': 0.0398625432}, 'time': '2020-04-28 15:49:27.302857'}
57274 done
57275
57275
{'emotion': {'Fear': 0.1104898452, 'Sad': 0.0372913109, 'Bored': 0.0818834175, 'Happy': 0.4586516217, 'Excited': 0.1871561004, 'Angry': 0.1245277043}, 'time': '2020-04-28 15:49:31.497352'}
57275 done
57276
57276
{'emotion': {'Fear': 0.1214678091, 'Sad': 0.3046319472, 'Bored': 0.0429197382, 'Happy': 0.1862339444, 'Excited': 0.0918135161, 'Angry': 0.252933045}, 'time': '2020-04-28 15:49:35.986017'}
57276 done
57277
57277
{'emotion': {'Fear': 0.0783382032, 'Sad': 0.0359547055, 'Bored': 0.1790042894, 'Happy': 0.2742191903, 'Excited': 0.1818714994, 'Angry': 0.2506121122}, 'time': '2020-04-28 15:49:41.393193'}
57277 done
57278
57278
{'emotion': {'Fear': 0.029803092, 'Sad': 0.0563884984, 'Bored': 0.019392117, 'Happy': 0.5287529556, 'Excited': 0.3456984747, 'Angry': 0.0199648622}, 'time': '2020-04-28 15:49:43.850863'}
57278 done
57279

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
with open('./lyrics_emotion_old.json', 'r') as f:
    emo1 = json.load(f)

In [4]:
len(emo1)

72626

In [14]:
with open('./lyrics_emotion.json', 'r') as f:
    emo2 = json.load(f)

In [15]:
len(emo2)

72852

In [7]:
emo2.update(emo1)

In [8]:
len(emo2)

72852

In [12]:
max(list(map(lambda x: int(x), list(emo2.keys()))))

73030

In [13]:
with open('./lyrics_emotion.json', 'w') as f:
    json.dump(emo2, f)